In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import plotly.offline as py
import plotly.graph_objs as go
from plotly import tools
import cufflinks as cf

py.init_notebook_mode()

In [ ]:
cf.set_config_file(offline=True, world_readable=False, theme='ggplot')

This module we'll be looking at the New York City tree census. This data was provided by a volunteer driven census in 2015, and we'll be accessing it via the socrata API. The main site for the data is [here](https://data.cityofnewyork.us/Environment/2015-Street-Tree-Census-Tree-Data/uvpi-gqnh), and on the upper right hand side you'll be able to see the link to the API.

The data is conveniently available in json format, so we should be able to just read it directly in to Pandas:

In [ ]:
url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json'
trees = pd.read_json(url)
trees.head(10)

Looks good, but lets take a look at the shape of this data:

In [ ]:
trees.shape

1000 seems like too few trees for a city like New York, and a suspiciously round number. What's going on?

Socrata places a 1000 row limit on their API. Raw data is meant to be "paged" through for applications, with the expectation that a UX wouldn't be able to handle a full dataset. 

As a simple example, if we had a mobile app with limited space that only displayed trees 5 at a time, we could view the first 5 trees in the dataset with the url below:

In [ ]:
firstfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=0'
firstfive_trees = pd.read_json(firstfive_url)
firstfive_trees

If we wanted the next 5, we would use this url:

In [ ]:
nextfive_url = 'https://data.cityofnewyork.us/resource/nwxe-4ae8.json?$limit=5&$offset=5'
nextfive_trees = pd.read_json(nextfive_url)
nextfive_trees

You can read more about paging using the Socrata API [here](https://dev.socrata.com/docs/paging.html)

In these docs, you'll also see more advanced functions (called `SoQL`) under the "filtering and query" section. These functions should be reminding you of SQL.

Think about the shape you want your data to be in before querying it. Using `SoQL` is a good way to avoid the limits of the API. For example, using the below query I can easily obtain the count of each species of tree in the Bronx:

In [ ]:
boro = 'Bronx'
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,count(tree_id)' +\
        '&$where=boroname=\'Bronx\'' +\
        '&$group=spc_common').replace(' ', '%20')
soql_trees = pd.read_json(soql_url)

soql_trees

This behavior is very common with web APIs, and I think this is useful when thinking about building interactive data products. When in a Jupyter Notebook or RStudio, there's an expectation that (unless you're dealing with truly large datasets) the data you want can be brought in memory and manipulated.

Dash and Shiny abstract away the need to distinguish between client side and server side to make web development more accessible to data scientists. This can lead to some unintentional design mistakes if you don't think about how costly your callback functions are (for example: nothing will stop you in dash from running a costly model triggered whenever a dropdown is called.)

The goal of using the Socrata is to force you to think about where your data operations are happening, and not resort to pulling in the data and performing all operations in local memory.

----------

**NOTE**: One tip in dealing with URLs: you may need to replace spaces with `'%20'`. I personally just write out the url and then follow the string with a replace:

In [ ]:
'https://api-url.com/?query with spaces'.replace(' ', '%20')

Now, let's make a query to get the count of trees per each unique combination of stewardship, health, species, and borough present in the data.

There are 5 levels of steward including empty (None, 1or2, 3or4, and 4orMore, and empty string), 4 levels of health (Poor, Fair, Good, and empty string), 133 levels of spc_common including empty string, and 5 boroughs. So we could have up to 5 * 4 * 133 * 5 = 13,300 results from this query. However, I checked in a browser and we get less than 1,000 results starting on offset=4000, meaning that there are less than 5,000 results. So run for no offset, then offset 1000, 2000, 3000, and 4000.

In [ ]:
soql_url = ('https://data.cityofnewyork.us/resource/nwxe-4ae8.json?' +\
        '$select=spc_common,boroname,health,steward,count(tree_id)' +\
        '&$group=spc_common,boroname,health,steward').replace(' ', '%20')

pg1 = pd.read_json(soql_url)
pg2 = pd.read_json(soql_url + '&$offset=1000')
pg3 = pd.read_json(soql_url + '&$offset=2000')
pg4 = pd.read_json(soql_url + '&$offset=3000')
pg5 = pd.read_json(soql_url + '&$offset=4000')

Combine all 5 pages of results into one Pandas data frame.

In [ ]:
counts = pd.concat([pg1,pg2,pg3,pg4,pg5])

Let's get results for species 'littleaf linden' in Queens.

In [ ]:
species = 'littleleaf linden'
boro = 'Queens'

counts_of_interest = counts[(counts['boroname'] == boro) & (counts['spc_common'] == species)]

counts_of_interest

Now, we need to visualize the proportion of trees at each health level for this species and borough. 

In [ ]:
counts_per_health = counts_of_interest.groupby('health').sum()['count_tree_id']

trace = go.Bar(
    x = counts_per_health.index,
    y = counts_per_health
)

layout = go.Layout(
    title = 'Species ' + species + ' in ' + boro,
    xaxis = dict(title = 'Health'),
    yaxis = dict(title = 'Number of trees')
)

fig = go.Figure(data = [trace], layout = layout)

py.iplot(fig)

Next, also visualize the percent of trees at each stewardship level with different health levels, and vice versa.

In [ ]:
counts_per_steward = counts_of_interest.groupby('steward').sum()['count_tree_id']

health_as_percent_steward = pd.merge(counts_of_interest,
                                     pd.DataFrame(counts_per_steward),
                                     on='steward')

health_as_percent_steward['percent_steward'] = health_as_percent_steward['count_tree_id_x']*100/health_as_percent_steward['count_tree_id_y']

steward_as_percent_health = pd.merge(counts_of_interest,
                                     pd.DataFrame(counts_per_health),
                                     on='health')

steward_as_percent_health['percent_health'] = steward_as_percent_health['count_tree_id_x']*100/steward_as_percent_health['count_tree_id_y']                                                                                                                    

In [ ]:
health_as_percent_steward = health_as_percent_steward.pivot(index='steward',
                                                            columns='health',
                                                            values='percent_steward')

steward_as_percent_health = steward_as_percent_health.pivot(index='health',
                                                            columns='steward',
                                                            values='percent_health')

In [ ]:
health_as_percent_steward.iplot(kind='bar', 
                                barmode='stack',
                                xTitle = 'Stewardship level',
                                yTitle = 'Percent of trees',
                                title = 'Species ' + species + ' in ' + boro)

steward_as_percent_health.iplot(kind='bar', 
                                barmode='stack',
                                xTitle = 'Health level',
                                yTitle = 'Percent of trees',
                                title = 'Species ' + species + ' in ' + boro)

Looks like the first plot is probably best, and we should also print raw numbers. We'll make something similar to what is in this document when we build the app.

Only thing we should probably do is re-level health to poor/fair/good. Let's do that.

Also make the plot title a bit more descriptive.

In [ ]:
health_as_percent_steward = health_as_percent_steward[['Poor','Fair','Good']]

health_as_percent_steward.iplot(kind='bar', 
                                barmode='stack',
                                xTitle = 'Stewardship level',
                                yTitle = 'Percent of trees',
                                title = 'Species ' + species + ' in ' + boro + '<br>Tree health as a function of stewardship')

Actually, one last thing to think about.

Did some exploratory analysis outside of this doc and found that species "green ash" in Staten Island has missing health information for one tree, while species "honeylocust" in Manhattan has missing health and steward information for one tree.

How do the plots look for these two instances?

In [ ]:
species = "green ash"
boro = "Staten Island"

counts_of_interest = counts[(counts['boroname'] == boro) & (counts['spc_common'] == species)]

counts_per_steward = counts_of_interest.groupby('steward').sum()['count_tree_id']

health_as_percent_steward = pd.merge(counts_of_interest,
                                     pd.DataFrame(counts_per_steward),
                                     on='steward')

health_as_percent_steward['percent_steward'] = health_as_percent_steward['count_tree_id_x']*100/health_as_percent_steward['count_tree_id_y']

health_as_percent_steward = health_as_percent_steward.pivot(index='steward',
                                                            columns='health',
                                                            values='percent_steward')

print(health_as_percent_steward)

health_as_percent_steward.iplot(kind='bar', 
                                barmode='stack',
                                xTitle = 'Stewardship level',
                                yTitle = 'Percent of trees',
                                title = 'Species ' + species + ' in ' + boro + '<br>Tree health as a function of stewardship')

health_as_percent_steward = health_as_percent_steward[['Poor','Fair','Good']]

health_as_percent_steward.iplot(kind='bar', 
                                barmode='stack',
                                xTitle = 'Stewardship level',
                                yTitle = 'Percent of trees',
                                title = 'Species ' + species + ' in ' + boro + '<br>Tree health as a function of stewardship')

In [ ]:
species = "honeylocust"
boro = "Manhattan"

counts_of_interest = counts[(counts['boroname'] == boro) & (counts['spc_common'] == species)]

counts_per_steward = counts_of_interest.groupby('steward').sum()['count_tree_id']

health_as_percent_steward = pd.merge(counts_of_interest,
                                     pd.DataFrame(counts_per_steward),
                                     on='steward')

health_as_percent_steward['percent_steward'] = health_as_percent_steward['count_tree_id_x']*100/health_as_percent_steward['count_tree_id_y']

health_as_percent_steward = health_as_percent_steward.pivot(index='steward',
                                                            columns='health',
                                                            values='percent_steward')

print(health_as_percent_steward)

health_as_percent_steward.iplot(kind='bar', 
                                barmode='stack',
                                xTitle = 'Stewardship level',
                                yTitle = 'Percent of trees',
                                title = 'Species ' + species + ' in ' + boro + '<br>Tree health as a function of stewardship')

health_as_percent_steward = health_as_percent_steward[['Poor','Fair','Good']]

health_as_percent_steward.iplot(kind='bar', 
                                barmode='stack',
                                xTitle = 'Stewardship level',
                                yTitle = 'Percent of trees',
                                title = 'Species ' + species + ' in ' + boro + '<br>Tree health as a function of stewardship')

Looks like when both are NA, it gets removed by the pivot. It is included when only health is NA. But for this one instance where the health NA makes up just .05% of the trees at that stewardship level, I think we can just ignore, and the bar only being 99.95 high instead of 100 should not even be noticeable.